##  MiniFlow

### Introduction 

A neural network is a graph of mathematical funstions such as linear combinations and activation functions. The graph consists of nodes, and edges.

Nodes in each layer(except for the input layer) perform mathematical functions using inputs from nodes in the previous layers. For example, a node could represent $f(x,y) = x + y$, where $x$ and $y$ are input values from nodes in the previous layer.

The edges in the graph describe the connection between the nodes, along which the values flow from one layer to the next.

MiniFlow is a neural network library meant to be a miniature version of Google's TensorFlow library. The library implements backpropogation and forward passing using a simple sigmoid activation function. 


### The Node class

In [2]:
class Node():
    def __init__(self, inbound_nodes=[]):
        # Node(s) from which this Node receives values
        self.inbound_nodes = inbound_nodes
        
        # Node(s) to which this Node passes values
        self.outbound_nodes = []
        
        # For each inbound Node here, add this Node as an outbound Node
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
        # A calculated value
        self.value = None
        
    def forward(self):
        """
        Forword propagation.
        
        Compute the output value based on 'inbound_nodes' and 
        store the result in self.value.
        """
        raise NotImplemented
        


### Input node class

Input nodes do no calculations but only hold values to be passed forward

In [3]:
class Input(Node):
    def __init__(self):
        # An Input node has no inbound nodes
        Node.__init__(self)
        
    def forward(self, value=None):
        # Overwrite the value if one is passed in
        if value is not None:
            self.value = value
            

### Add node class

Takes 2 inbound nodes, x, and y, and adds the values of those nodes.

In [4]:
class Add(Node):
    def __init__(self, x, y):
        Node.__init__(self, [x,y])
    
    def forward(self):
        """
        Calculates sum and passes it forward
        """